In [ ]:
#필요한 것들 install
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U
!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install -U langchain-huggingface
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
!pip install langchain-teddynote
!pip install peft

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

import random
import gc
from transformers import (
    AutoModel,
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    set_seed,
    TrainingArguments, 
    Trainer
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

from torch.nn.parallel import DataParallel

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_huggingface import HuggingFacePipeline
from langchain.embeddings.base import Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from datasets import Dataset
from functools import partial
from torch.cuda.amp import GradScaler
from IPython.display import FileLink, display
from huggingface_hub import login
import wandb
import subprocess
torch.utils.checkpoint.use_reentrant = False
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bitsandbytes")

In [ ]:
test_source = [
    "「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》",
    "「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》",
    "「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》",
    "「FIS 이슈&포커스」 22-2호 《재정성과관리제도》",
    "국토교통부_행복주택출자",
    "보건복지부_노인장기요양보험 사업운영",
    "보건복지부_부모급여(영아수당) 지원",
    "산업통상자원부_에너지바우처",
    "중소벤처기업부_혁신창업사업화자금(융자)"
]

train_source = [
    "「FIS 이슈 & 포커스」 22-3호 《재정융자사업》",
    "「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》",
    "1-1 2024 주요 재정통계 1권",
    "2024 나라살림 예산개요",
    "2024년도 성과계획서(총괄편)",
    "고용노동부_내일배움카드(일반)",
    "고용노동부_조기재취업수당",
    "고용노동부_청년일자리창출지원",
    "국토교통부_민간임대(융자)",
    "국토교통부_소규모주택정비사업",
    "국토교통부_전세임대(융자)",
    "보건복지부_노인일자리 및 사회활동지원",
    "보건복지부_생계급여",
    "월간 나라재정 2023년 12월호",
    "재정통계해설",
    "중소벤처기업부_창업사업화지원"
]
# 두 리스트를 합칩니다
all_sources = test_source + train_source

# 딕셔너리를 생성합니다
h2n = {name : str(idx) for idx, name in enumerate(all_sources)}
n2h = {str(idx) : name for idx, name in enumerate(all_sources)}

for i in range(len(all_sources)):
    print(i,n2h[str(i)])


In [ ]:
#선택할 옵션들, 여기를 수정하세요
class Opt:
    def __init__(self):
        self.llm_model = "/kaggle/input/llama-3-1-8b-instruct" #based on yours, 미리 다운받고 로컬 주소를 입력해도 됨(다운 시간 절약) , meta-llama/Meta-Llama-3.1-8B-Instruct
        self.embeddings_model = 'intfloat/multilingual-e5-base' #based on yours, 미리 다운받고 로컬 주소를 입력해도 됨(다운 시간 절약) ,intfloat/multilingual-e5-large
        self.rerank_model = 'cross-encoder/ms-marco-MiniLM-L-6-v2' #based on yours ,미리 다운받고 로컬 주소를 입력해도 됨(다운 시간 절약) 
        self.base_directory = "/kaggle/input/rag-data" #based on yours
        self.output_dir = "/kaggle/working/results" #based on yours
        
        os.makedirs(self.output_dir, exist_ok=True) 
        #없으면 디렉토리 생성
        self.hf_token = '' #based on yours
        #Jupyter Notebook에서 작업했기 떄문에 직접 입력 
        
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.submission_csv_path = os.path.join(self.base_directory, "sample_submission.csv")
        self.results_path = os.path.join(self.output_dir, "submission.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32

        self.rerank = False #rerank 
        self.train = False #Train으로 학습 여부
        self.trained_model_path = "/kaggle/working/finetuned_model" 
        if self.train:
            os.makedirs(self.trained_model_path, exist_ok=True) 
            #없으면 디렉토리 생성
        self.peft = False # 학습된 모델 사용여부
        self.finetuned_data='/kaggle/input/finetuned-dataset/finetuned_dataset.json'
        self.template = """
Provide an answer to the question based on the following information.
When responding:

Focus on the core of the question.
Keep the response concise, within 1–2 sentences.
Use the units provided in the document.
Ensure the answer is polite and in a complete form, including the subject of the question.
Additionally, the information and question will be in Korean, and the response should also be written in Korean.

Information:
{context}

Question:
{question}

Answer:

<|eot_id|>
"""
# 다음 정보를 바탕으로 질문에 답하세요.
# 이때 질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고,
# 단위는 문서에서 나온 단위를 사용하며,
# 질문의 주어를 포함해 완성된 존댓말로 답변해주세요.:

# ### 정보: 
# {context}

# ### 질문:
# {question}

# ### 답변:

# <|eot_id|>
# """
            #<|eot_id|>는 llama3.1 Instruct모델에서 사용한 특수 토큰으로 사용시 답변이 길어지는것을 방지함, 모델이 바뀐다면 확인해야함
args=Opt()

# Hugging Face 로그인 , 미리 해당 모델의 사용 권환을 승인 받아야함
hf_token = args.hf_token
login(hf_token)

wandb.login(key ='') 
wandb.init(project="dacon_project")  # 프로젝트 이름 설정


In [ ]:
import gc
import psutil
import os
def get_memory_usage():
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # MB로 변환
    return f"현재 메모리 사용량: {memory_usage:.2f} MB"
    
print(get_memory_usage())

In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    #PDF 텍스트 추출 후 chunk 단위로 나누기
    #PDF 파일 열기
    for i in h2n:
        if i in file_path:
            file_path = file_path.replace(i,h2n[i])
            break
            
    doc = fitz.open(file_path)
    text = ''
    
    for page in doc:
        text += page.get_text()
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    chunks = [Document(page_content=t) for t in chunk_temp]
    doc.close()
    return chunks

def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db
    
    
def normalize_string(string):
    #유니코드 정규화
    return unicodedata.normalize('NFC', string)
    
def process_pdfs_from_dataframe(df, base_directory):
    #PDF정보로 Retreiver생성
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        normalized_path = normalize_string(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        chunks = process_pdf(full_path,args.chunk_size,args.chunk_overlap)
        db = create_vector_db(chunks,args.embeddings_model)
                
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()
 
        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[0.5,0.5],
            search_type="mmr",
        )
        
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
        del chunks, db, kiwi_bm25_retriever, faiss_retriever, retriever
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

        
    return pdf_databases
        
def get_combined_docs(docs, question):
    reranker = CrossEncoder(args.rerank_model)
    
    pairs = [[question, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    
    scored_docs = list(zip(docs, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    top_docs = [doc.page_content for doc, _ in scored_docs]
    return "\n\n".join(top_docs)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    print(len(docs))
    for doc in docs:
        context += doc.page_content
        context += '\n'
    # print(context)
    # print()
    return context

In [ ]:
#GPU초기화 ,환경은 유지되는대신 모든 변수들은 초기화됨

#!for pid in $(lsof /dev/nvidia* | awk '{print $2}' | tail -n +2); do kill -9 $pid; done

#nvida gpu 사용 프로세스 확인
print(get_memory_usage())

In [ ]:
import json
def save_dataset(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

def load_dataset(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        dataset_list = json.load(f)
    return dataset_list

In [ ]:
if args.train:
        
    def prepare_finetune_data(df, pdf_databases):
        finetune_data = []
        for _, row in df.iterrows():
            source = normalize_string(row['Source'])
            question = row['Question']
            answer = row['Answer']  # 실제 답변이 있다고 가정
            
            retriever = pdf_databases[source]['retriever']
            docs = retriever.invoke(question)
            context = get_combined_docs(docs, question)
            
            prompt = args.template.format(context=context, question=question)
            finetune_data.append({
                "instruction": prompt,\
                "output": answer
            })
        return finetune_data
        
    def preprocess_function(examples):
        inputs = tokenizer(examples["instruction"], truncation=True, padding="max_length", max_length=512)
        outputs = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=512)
        inputs["labels"] = outputs["input_ids"]
        return inputs
        
    if args.finetuned_data :
        print('Data 다운 완료')
        finetune_dataset = load_dataset(args.finetuned_data)
    else:
        train_df = pd.read_csv(args.train_csv_path)
        train_pdf_databases = process_pdfs_from_dataframe(train_df, args.base_directory)
        finetune_dataset = prepare_finetune_data(train_df, train_pdf_databases)
        save_dataset(finetune_dataset, '/kaggle/working/finetune_dataset.json')
    print('Data 준비 완료')
    model_name =args.llm_model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # 8비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    
    num_gpus = torch.cuda.device_count()
    max_memory = {i: "14GiB" for i in range(num_gpus)}

    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=bnb_config,
        device_map="balanced",
        max_memory=max_memory,
        trust_remote_code=True
    )
    model = prepare_model_for_kbit_training(model)
    
    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # LoRA 모델 생성
    model = get_peft_model(model, peft_config)
    print('모델 준비 완료')
    
    dataset = Dataset.from_pandas(pd.DataFrame(finetune_dataset))

    tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)
    print('tokenized 완료')
    training_args = TrainingArguments(
        output_dir=args.trained_model_path,
        num_train_epochs=10,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        learning_rate=5e-5,
        fp16=True,
        save_steps=100,
        logging_steps=50,
        optim="paged_adamw_8bit",
        report_to="none",  # wandb 로깅 활성화
        remove_unused_columns=False,
        gradient_checkpointing=True
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset
    )
    print('Train 시작')
    trainer.train()

    model.save_pretrained(args.trained_model_path)
    tokenizer.save_pretrained(args.trained_model_path)

In [ ]:
df = pd.read_csv(args.test_csv_path)
pdf_databases = process_pdfs_from_dataframe(df, args.base_directory)       

In [ ]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False

    num_gpus = torch.cuda.device_count()
    max_memory = {i: "14GiB" for i in range(num_gpus)}
    #멀티 GPU할당 Kaggle T4 x 2 이기 때문에 하나에 14GB할당
    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=BitsAndBytesConfig(
                load_in_8bit=True  # 8-bit 양자화 활성화
            ),
        torch_dtype="auto",
        device_map="balanced",
        max_memory=max_memory,
        trust_remote_code=True
    )
    if args.peft:
        model = PeftModel.from_pretrained(model, args.trained_model_path)
        model = model.merge_and_unload()
        print('PEFT!')
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

llm = setup_llm_pipeline()

In [ ]:
results = []
if args.rerank: print('rerank 적용')
else:           print('rerank 미적용')
    
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    source = normalize_string(row['Source'])
    question = row['Question']

    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    prompt = PromptTemplate.from_template(args.template) 

    if args.rerank:
        combine_with_question = partial(get_combined_docs,question=question)
    else:
        combine_with_question = format_docs
        
    rag_chain = (
        {"context": retriever | combine_with_question, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)
    print(f"Answer: {full_response}\n")
        
    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response

    })

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(args.submission_csv_path)
# 생성된 답변에서 앞뒤의 공백 및 줄바꿈 제거 후 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'].strip() for item in results]  # strip()으로 앞뒤 공백 제거
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")  # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [주의]

print(submit_df.head(3))
submit_df.to_csv(args.results_path, encoding='UTF-8-sig', index=False)

In [ ]:
#결과 다운 코드 생성
def download_file(path):
    os.chdir(os.path.dirname(path))
    
    command = f"zip -r output.zip {os.path.basename(path)}/"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink('output.zip'))
    
#download_file(args.output_dir)
download_file(args.output_dir)